In [2]:
!pip install numpy
!pip install sanssouci
!pip install unittest

ERROR: Could not find a version that satisfies the requirement unittest (from versions: none)
ERROR: No matching distribution found for unittest


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import unittest
from sanssouci import curve_max_fp, find_largest_region

In [49]:
def interpolation_naif(p_values: list, thresholds: list, zeta: list=[k for k in range(5000)]) -> int:
    "Naive calculus posthoc bounds"
    p_values = np.sort(p_values)
    K = len(thresholds)
    kmax = min(len(p_values), K)
    B = []
    for k in range(kmax):
        compteur = 0
        for p_value in p_values:
            if p_value >= thresholds[k]:
                compteur += 1
        B.append(compteur + zeta[k])
    if min(B) <= len(p_values):
        print("La borne correspond au k :", np.argmin(B))
    else:
        print("Nous avons la borne triviale")
    return min(min(B), len(p_values))

In [50]:
def interpolation(p_values: list, thresholds: list, zeta: list=[k for k in range(5000)]) -> int:
    "Calcul posthoc bounds"
    p_values = np.sort(p_values)
    K = len(thresholds)
    kmax = min(len(p_values), K)
    B = []
    i_start = 0
    for k in range(kmax):
        i = i_start
        while i < len(p_values) and p_values[i] < thresholds[k]:
            i += 1
            i_start = i
        B.append(len(p_values) - i + zeta[k]) # k est un cas particulier de Zeta_k
    if min(B) <= len(p_values):
        print("La borne correspond au k", np.argmin(B))
    else:
        print("Nous avons la borne triviale")
    return min(min(B), len(p_values))

Quelle est la complexité de cet algorithme?
Trier la liste des p-valeurs : O(slog(s))
Parcourir la liste des indices k : O(k)
Parcourir la liste des p-valeurs : O(log(s))
Chercher le minimum dans la liste ?

In [51]:
def interpolation_bis(p_values: list, thresholds:list, zeta: list=[k for k in range(5000)]) -> int:
    p_values = np.sort(p_values)
    s = len(p_values)
    K = len(thresholds)
    B = []
    k, i = 0, 0
    while k < K and i < s:
        if p_values[i] < thresholds[k]:
            i += 1
        else:
            B.append(s - i + zeta[k])
            k += 1
    B.append(s - i + zeta[k])
    if min(B) < len(p_values):
            print("La borne correspond au k :", np.argmin(B))
    else:
        print("Nous avons la borne triviale")
    return min(min(B), len(p_values))


In [52]:
def interpolation_minmax_naif(p_values: list, thresholds: list, kmin:int, kmax: int, zeta: list=[k for k in range(5000)]) -> int:
    "Naive calculus of the posthoc bounds with kmin and kmax"
    p_values = np.sort(p_values)
    B = []
    if kmin == kmax:
        return len(p_values)
    else:
        for k in range(kmin, kmax):
            compteur = 0
            for p_value in p_values:
                if p_value >= thresholds[k]:
                    compteur += 1
            B.append(compteur + zeta[k])
        if min(B) <= len(p_values):
            print("La borne correspond au k :", np.argmin(B))
        else:
            print("Nous avons la borne triviale")
        return min(min(B), len(p_values))

In [53]:
def interpolation_minmax(p_values: list, thresholds: list, kmin: int, kmax: int, zeta: list= [k for k in range(5000)]) -> int:
    "La bound posthoc est calculée pour les k compris entre kmin et kmax"
    p_values = np.sort(p_values)
    B = []
    i_start = 0
    if kmin == kmax:
        i = 0
        while i < len(p_values):
            if p_values[i] < thresholds[kmax - 1]:
                i += 1
            else:
                return min(len(p_values) - i + zeta[kmax - 1], len(p_values))
    else:
        for k in range(kmin, min(kmax, len(p_values))):
            i = i_start
            while i < len(p_values) and p_values[i] < thresholds[k]:
                    i += 1
                    i_start = i
            B.append(len(p_values) - i + zeta[k]) # k est un cas particulier de Zeta_k
        if min(B) < len(p_values):
            print("La borne correspond au k :", np.argmin(B))
        else:
            print("Nous avons la borne triviale")
        return min(min(B), len(p_values))

In [54]:
m = 1000
alpha = 0.9
K = 500

np.random.seed(23)
values = np.linspace(0, 1, 300)
# p_values = values ** 2
p_values = [np.random.beta(a=k, b=m-k+1) for k in range(1, K)]
thresholds = np.array([alpha*k/m for k in range(K)])

In [55]:
interpolation_minmax(p_values, thresholds, kmin=0, kmax=K)

La borne correspond au k : 2


497

In [71]:
def linear_interpolation(p_values : list, thresholds: list, kmin: int=0)-> list:
    p_values = np.sort(p_values)
    s = len(p_values)
    K = len(thresholds)
    tau = np.zeros(s)
    for k in range(s):
        if k < kmin:
            tau[k] = 0
        elif kmin <= k < K:
            tau[k] = thresholds[k]
        else:
            tau[k] = thresholds[K-1]
    kappa = np.ones(s, dtype=int)*s
    r = np.ones(s, dtype=int)*s
    k, i = 0, 0
    while k < s and i < s:
        if p_values[i] < tau[k]:
            kappa[i] = k
            i += 1
        else:
            r[k] = i
            k += 1
    V, A, M = np.zeros(s), np.zeros(s), np.zeros(s)
    M[0] = r[0]
    for k in range(s):
        A[k] = r[k] - k
        if k > 0:
            M[k] = max(M[k-1], A[k])
    Z = np.zeros(s)
    indice = 0
    value = M[0]
    for i in range(s):
        if M[i] < value:
            value = M[i]
            indice = i
        Z[i] = indice
    print("salut", Z)
    for i in range(s):
        if kappa[i] > 0:
            V[i] = int(min(kappa[i], i+1 - M[kappa[i]-1]))
            if V[i] == kappa[i]:
                Z[i] = None
    print("Les argmin sont :" )
    return V

In [72]:
linear_interpolation(p_values, thresholds)

salut [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

array([  1.,   1.,   2.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,
        10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,
        21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,
        32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,
        43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,
        54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,
        65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,
        76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,
        87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,
        98.,  99., 100., 101., 102., 103., 104., 105., 106., 107., 108.,
       109., 110., 111., 112., 113., 114., 115., 116., 117., 118., 119.,
       120., 121., 122., 123., 124., 125., 126., 127., 128., 129., 130.,
       131., 132., 133., 134., 135., 136., 137., 138., 139., 140., 141.,
       142., 143., 144., 145., 146., 147., 148., 14

In [11]:
def interpolation_segments(p_values: list, thresholds: list, L: list)-> int:
    "La bound posthoc est calculée pour les k entre les (kmin, kmax)"
    return min([interpolation_minmax(p_values, thresholds, kmin= element[0], kmax=element[1]) for element in L])

In [17]:
def linear_interpolation_zeta(p_values: list, thresholds: list, zeta, kmin: int=0) -> list:
    p_values = np.sort(p_values)
    s = len(p_values)
    K = len(thresholds)
    K = max(s, K)
    tau = np.zeros(K)
    for k in range(K):
        if k < kmin:
            tau[k] = 0
        elif kmin <= k < len(thresholds):
            tau[k] = thresholds[k]
        else:
            tau[k] = thresholds[len(thresholds)-1]
    kappa = np.ones(K, dtype=int)*K
    r = np.ones(K, dtype=int)*s
    k, i = 0, 0
    while k < K and i < s:
        if p_values[i] < tau[k]:
            kappa[i] = k
            i += 1
        else:
            r[k] = i
            k += 1
            
    k, j = 0, 0
    z = np.ones(K, dtype=int)*K
    while k < K and j < s:
        if zeta[k] >= j+1:
            z[j] = k-1
            j += 1
        else:
            k += 1
    V, A, M = np.zeros(s), np.zeros(K), np.zeros(K)
    M[0] = r[0]
    for k in range(K):
        A[k] = r[k] - zeta[k]
        if k > 0:
            M[k] = max(M[k-1], A[k])
    for i in range(s):
        if kappa[i] > 0:
            V[i] = int(min(zeta[kappa[i]], i+1 - M[kappa[i]-1]))
    return V

In [30]:
m = 10000
alpha = 0.9
K = 600
s = 800
# np.random.seed(42)
values = np.linspace(0, 1, s)
# p_values = values ** 2
# p_values = [1]*s
# p_values = [np.random.beta(a=k, b=m-k+1) for k in range(1, s+1)]
thresholds = np.array([alpha*k/m for k in range(K)])
# thresholds = np.sort(np.random.uniform(low=0, high=1, size=K))

In [14]:
zeta = [k**4 for k in range(5000)]

In [69]:
M = np.array([4, 4, 4, 2, 2, 1, 1, 0])

L = []
valeur = M[0]
indice = 0
for k in range(len(M)):
    if M[k] < valeur:
        valeur = M[k]
        indice = k
    L.append(indice)
L

[0, 0, 0, 3, 3, 5, 5, 7]